In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('stopwords')
import os
from time import time  # to time our operations

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
def stopwords_city(City):
    month = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december' ]
    day = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
    if City == 'Boston':
        return month + day + ['boston', 'roxbury','fiandaca','opencounter', 'accela','forth', 'ortega', 'burro','theroxburyinnovation', 'julie', 'elliott', '21stcentury', 'nstar','dorsey','austin', 'blackmon', 'swett', 'ofboston', 'paired','ass','first', 'martin', 'addition', 'every', 'walsh', 'according', 'please visit', 'noted',
            'please','april', 'february', 'visit', 'across', 'december', 'https', 'following',
             'noted', 'along', 'three', 'total', 'pointed', 'citys', 'announced', 'massachusetts', 'million']
    if City =='Taipei':
        return month + day +['taipei','since', 'dadaocheng', 'serf', 'addition', 'february','related', 'october', 'january', 'around', 'taking', 'march', 'first', 'according', 'please visit',
                'noted', 'please','april', 'among''february', 'visit', 'across', 'december', 'https', 'ko',
                'following', 'universiade','september','november','august','noted', 'along', 'three', 'total', 'pointed', 'citys', 'chinese']
    if City == 'Seoul':
        return month + day +['seoul','yeomchang', 'dulle','three', 'gaehwasinnonhyeon', 'major', 'first', 'million', 'various','selected','around','number', 'based','total',
                'order','special', 'since']
    if City == 'Helsinki':
        return month + day + ['helsinki','based', '6aika', 'wrtsil','january', 'first', 'sitra']

In [4]:
def corpus(City):
    folder = 'C:/Users/user/PycharmProjects/DataCamp/{}_Data'.format(City)

    txt = ''
    stopwords = nltk.corpus.stopwords.words('english')
    stopwords.extend(stopwords_city(City))
    corpus1 = []
    tokenizer = RegexpTokenizer(r'\w+')
    for file in os.listdir(folder):
        path = os.path.join(folder, file)
        book = open(path, 'r')
        text = book.read()
        #data = text.split('.')
        data = text.splitlines()

        for sentence in data:
            shortword = re.compile(r'\W*\b\w{1,4}\b')
            sentence = shortword.sub('', sentence).lower()
            word_list = tokenizer.tokenize(sentence.lower())
            word_list1 = [word for word in word_list if word.isalpha()]
            word_list2 = [word for word in word_list1 if word not in stopwords]
            corpus1.append(word_list2)

    # Build the bigram and trigram models
    bigram = gensim.models.Phrases(corpus1, min_count=5, threshold=100)  # higher threshold fewer phrases.
    trigram = gensim.models.Phrases(bigram[corpus1], threshold=100)

    # Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)

    def make_bigrams(texts):
        return [bigram_mod[doc] for doc in texts]

    def make_trigrams(texts):
        return [trigram_mod[bigram_mod[doc]] for doc in texts]

    corpus2 = make_bigrams(corpus1)

    #print(corpus2[:10])

    lemmatizer = WordNetLemmatizer()
    for w in range(len(corpus2)):
        x = 0
        while x <(len(corpus2[w])):
            corpus2[w][x] = lemmatizer.lemmatize(corpus2[w][x])
            if corpus2[w][x] in stopwords:
                del corpus2[w][x]
            x += 1

    return corpus2

In [5]:
t = time()

City = 'Boston' ##Input
data_lemmatized = corpus(City)
id2word = corpora.Dictionary(data_lemmatized)
id2word.save('dictionary.gensim.{}'.format(City))
texts = data_lemmatized
corpus = [id2word.doc2bow(text) for text in texts]

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                id2word=id2word,
                                                num_topics=20,
                                                random_state=100,
                                                update_every=1,
                                                chunksize=100,
                                                passes=10,
                                                alpha='auto',
                                                per_word_topics=True)

lda_model.save('model3.gensim.{}'.format(City))
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

print('Perplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)


print('Time to process {} data: {} mins'.format(City, round((time() - t) / 60, 2)))

[(0,
  '0.081*"health" + 0.065*"service" + 0.037*"shelter" + 0.030*"homeless" + '
  '0.029*"housing" + 0.028*"recovery" + 0.021*"individual" + '
  '0.020*"prevention" + 0.018*"homelessness" + 0.017*"emergency"'),
 (1,
  '0.112*"housing" + 0.056*"development" + 0.039*"affordable" + 0.034*"unit" + '
  '0.020*"income" + 0.017*"neighborhood" + 0.013*"building" + 0.013*"senior" + '
  '0.012*"property" + 0.012*"cost"'),
 (2,
  '0.031*"resident" + 0.023*"department" + 0.013*"parking" + '
  '0.013*"transportation" + 0.012*"neighborhood" + 0.011*"safety" + '
  '0.010*"process" + 0.010*"information" + 0.008*"mayor" + 0.008*"permit"'),
 (3,
  '0.062*"energy" + 0.061*"city" + 0.043*"climate" + 0.039*"election" + '
  '0.038*"action" + 0.027*"building" + 0.025*"climate_change" + 0.025*"voter" '
  '+ 0.019*"summit" + 0.017*"environment"'),
 (4,
  '0.052*"finding" + 0.030*"concert" + 0.022*"james" + 0.020*"latin" + '
  '0.017*"patriot" + 0.017*"patient" + 0.017*"incident" + 0.015*"hospital" + '
  '0.0

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os 
%matplotlib inline
import pyLDAvis.gensim
import gensim
pyLDAvis.enable_notebook()

d = gensim.corpora.Dictionary.load('dictionary.gensim.Boston')
c = [id2word.doc2bow(text) for text in texts]
lda = gensim.models.LdaModel.load('model3.gensim.Boston')

pd.options.display.max_colwidth = 5000
data = pyLDAvis.gensim.prepare(lda, c, d, mds='tsne')
pyLDAvis.show(data)

In [9]:
pyLDAvis.save_html(data, 'Boston.20.lda.html')